In [1]:
#from __future__ import print_function
#from __future__ import absolute_import
#from __future__ import division


import torch 
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dataset
import visdom
import sys,argparse

from models import scribbler, discriminator
import torch.optim as optim

from skimage import color
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
import visualize
import torch.nn as nn
from torch.autograd import Variable
from IPython.display import display
import torchvision.models as models
from dataloader import imfol

from utils.visualize import vis_patch, vis_image
        
from torch.utils.data import DataLoader
from dataloader.imfol import ImageFolder, make_dataset
from utils import transforms as custom_trans
import torchvision.transforms as tforms
import utils.transforms as utforms

from networks import define_G, weights_init

import visdom

In [2]:
def clamp_image(img):
    img[:,0,:,:].clamp_(0,1)
    img[:,1,:,:].clamp_(-1.5,1.5)
    img[:,2,:,:].clamp_(-1.5,1.5)
    return img    

In [3]:
def parse_arguments(argv):
    parser = argparse.ArgumentParser()
    
###############added options#######################################
    parser.add_argument('-learning_rate', default=1e-4, type=float,
                    help='Learning rate for the generator')
    parser.add_argument('-learning_rate_D',  default=1e-4,type=float,
                    help='Learning rate for the discriminator')    
    
    parser.add_argument('-gan', default='dcgan',type=str,choices=['dcgan', 'lsgan'],
                    help='dcgan|lsgan') #todo wgan/improved wgan    
    
    parser.add_argument('-num_epoch',  default=1,type=int,
                    help='texture|scribbler')   
    
    parser.add_argument('-visualize_every',  default=10,type=int,
                    help='no. iteration to visualize the results')      

    #all the weights ratio, might wanna make them sum to one
    parser.add_argument('-feature_weight', default=1,type=float,
                       help='weight ratio for feature loss')
    parser.add_argument('-pixel_weight_l', default=0,type=float,
                       help='weight ratio for pixel loss for l channel')
    parser.add_argument('-pixel_weight_ab', default=0,type=float,
                   help='weight ratio for pixel loss for ab channel')
    parser.add_argument('-tv_weight', default=1,type=float,
                   help='weight ratio for total variation loss')
    parser.add_argument('-discriminator_weight', default=0,type=float,
                   help='weight ratio for the discriminator loss')
    parser.add_argument('-style_weight', default = 0, type=float, 
                        help='weight ratio for the texture loss')

    parser.add_argument('-gpu', default=1,type=int,
                   help='id of gpu to use, -1 for cpu')

    parser.add_argument('-display_port', default=7779,type=int,
               help='port for displaying on visdom (need to match with visdom currently open port)')

    parser.add_argument('-data_path', default='/home/psangkloy3/training_handbags_pretrain/',type=str,
                   help='path to the data directory, expect train_skg, train_img, val_skg, val_img')


############################################################################
############################################################################
############TODO: TO ADD#################################################################
    parser.add_argument('-content_layers',  default='relu2_2',type=str,
                    help='Layer to attach content loss.')
    
    parser.add_argument('-batch_size', default=1) #fixed batch size 1
    
    parser.add_argument('-image_size',default=128,type=int,
                    help='Training images size, after cropping')        
    parser.add_argument('-resize_max',  default=256,type=int,
                    help='max resize size')        
    parser.add_argument('-resize_min',  default=128,type=int,
                    help='min resize size')   
    

    
    parser.add_argument('-mode',  default='texture',type=str,choices=['texture','scribbler'],
                    help='texture|scribbler') 
    
    parser.add_argument('-save_every',  default=50000,type=int,
                    help='no. iteration to save the models')
    
    parser.add_argument('-crop',  default='random',type=str,choices=['random','center'],
                    help='random|center')
    
    parser.add_argument('-contrast',  default=True,type=bool,
                    help='randomly adjusting contrast on sketch')
    
    parser.add_argument('-occlude', default=False,type=bool,
                       help='randomly occlude part of the sketch')
    
    
    parser.add_argument('-checkpoints_path', default='data/',type=str,
                   help='output directory for results and models')
    
    parser.add_argument('-model', default='scribbler_custom',type=str,
                   help='generator architecture')
    
    parser.add_argument('-noise_gen', default=False,type=bool,
                   help='whether or not to inject noise into the network')
    
    parser.add_argument('-load', default=1,type=int,
                   help='load generator and discrminator from iteration n')
    parser.add_argument('-load_D', default=1,type=float,
                   help='load discriminator from iteration n, priority over load')
    
    parser.add_argument('-absolute_load', default='',type=str,
                   help='load saved generator model from absolute location')
    
    
    
    
##################################################################################################################################    
    
    return parser.parse_args(argv)

In [4]:
#command = '-content_layers relu2_2 -feature_weight 100 -pixel_weight_ab 1 -tv_weight 0.0001 -model scribbler_custom -data_path /home/psangkloy3/training_handbag_pretrain/ -gpu 1 -display_port 7779 -image_size 128 -save_every 5000 -visualize_every 10 -discriminator_weight 0 -learning_rate 1e-3 -learning_rate_D 1e-6 -batch_size 6 -contrast True -resize_max 256 -resize_min 128 -gan lsgan -load 100000'
command = '-content_layers relu2_2 -feature_weight 100 -style_weight 1 -pixel_weight_ab 1 -pixel_weight_l 1 -tv_weight 0.0001 -model scribbler_custom -data_path /home/wxian3/training_handbags_pretrain/ -gpu 3 -display_port 7776 -image_size 128 -save_every 5000 -visualize_every 10 -discriminator_weight 0 -learning_rate 1e-3 -learning_rate_D 1e-6 -batch_size 6 -contrast True -resize_max 256 -resize_min 128 -gan lsgan'
argv = parse_arguments(command.split())

In [5]:
class GramMatrix(nn.Module):

    def forward(self, input):
        a, b, c, d = input.size()  # a=batch size(=1)
        # b=number of feature maps
        # (c,d)=dimensions of a f. map (N=c*d)

        features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL

        G = torch.mm(features, features.t())  # compute the gram product

        # normalize the values of the gram matrix
        # by dividing by the number of element in each feature maps.
        return G.div(a * b * c * d)

In [6]:
class FeatureExtractor(nn.Module):
    # Extract features from intermediate layers of a network

    def __init__(self, submodule, extracted_layers):
        super(FeatureExtractor,self).__init__()
        self.submodule = submodule
        self.extracted_layers=extracted_layers

    def forward(self, x):
        outputs = []
        for name, module in self.submodule._modules.items():
            x = module(x)
            if name in self.extracted_layers:
                outputs += [x]
        return outputs + [x]
    

In [ ]:

with torch.cuda.device(argv.gpu):
    
    vis=visdom.Visdom(port=argv.display_port)
    
    Loss_g_graph=[]
    Loss_gd_graph=[]
    Loss_gf_graph = []
    # style loss graph
    Loss_gs_graph = []
    Loss_gp_graph = []
    Loss_d_graph=[]

    ts=tforms.Compose([custom_trans.toLAB(), custom_trans.toTensor()])
    rgbify = custom_trans.toRGB()
    dset = ImageFolder(argv.data_path,ts)
    dataloader=DataLoader(dataset=dset, batch_size=2, shuffle=True)
    
    sigmoid_flag = 1
    if argv.gan =='lsgan':
        sigmoid_flag = 0 
        

    netG=define_G(3,3,32)
    netD=discriminator.Discriminator(3,32,sigmoid_flag)  
    feat_model=models.vgg19(pretrained=True)

    netG.apply(weights_init)
    netD.apply(weights_init)    

    
    if argv.gan =='lsgan':
        criterion_gan = nn.MSELoss()
    elif argv.gan =='dcgan':
        criterion_gan = nn.BCELoss()
        
    criterion_l1 = nn.L1Loss()
    
    criterion_feat = nn.L1Loss()
    
    

    input_skg = torch.FloatTensor(2, 3, 256, 256)
    output_img = torch.FloatTensor(2, 3, 256, 256)
    label = torch.FloatTensor(2)
    real_label = 1
    fake_label = 0

    optimizerD = optim.Adam(netD.parameters(), lr=argv.learning_rate_D, betas=(0.5, 0.999))
    optimizerG = optim.Adam(netG.parameters(), lr=argv.learning_rate, betas=(0.5, 0.999))

    netG.cuda()
    netD.cuda()
    feat_model.cuda()
    criterion_gan.cuda()
    criterion_l1.cuda()
    criterion_feat.cuda()
    input_skg, output_img, label = input_skg.cuda(), output_img.cuda(), label.cuda()


    for epoch in range(argv.num_epoch):
        for i, data in enumerate(dataloader, 0):
            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            ###########################
            # train with real
            netD.zero_grad()
            img, skg = data
            img=utforms.normalize_lab(img)
            skg=utforms.normalize_lab(skg)
            img=img.cuda()
            skg=skg.cuda()
            input_skg.resize_as_(skg.float()).copy_(skg)
            output_img.resize_as_(img.float()).copy_(img)
            inputv = Variable(input_skg)
            outputv = Variable(output_img)
            labelv = Variable(label)
            #print labelv.data.size()

            output = netD(inputv)
            
            label.resize_(output.data.size())
            labelv = Variable(label.fill_(real_label))
            errD_real = criterion_gan(output, labelv)
            errD_real.backward()
            D_x = output.data.mean()

            # train with fake
            #noise.resize_(batch_size, nz, 1, 1).normal_(0, 1)
            #noisev = Variable(noise)
            fake = netG(inputv)
            output = netD(fake.detach())
            label.resize_(output.data.size())
            labelv = Variable(label.fill_(fake_label))

            errD_fake = criterion_gan(output, labelv)
            errD_fake.backward()
            D_G_z1 = output.data.mean()
            errD = errD_real + errD_fake
            Loss_d_graph.append(errD.data[0])
            optimizerD.step()

            ############################
            # (2) Update G network: maximize log(D(G(z)))
            ###########################
            netG.zero_grad()
            labelv = Variable(label.fill_(real_label))  # fake labels are real for generator cost
            output = netD(fake)

            multer=torch.ones(inputv.data.size())
            multer[:,1:,:,:]=argv.pixel_weight_ab*multer[:,1:,:,:]
            multer=multer.cuda()
            multer=Variable(multer)
            
            multer_=torch.ones(inputv.data.size())
            multer_[:,0,:,:]=argv.pixel_weight_l*multer_[:,0,:,:]
            multer_=multer_.cuda()
            multer_=Variable(multer_)
            
            #print outputv.size(), multer.size()
            woutputv = outputv*multer*multer_
            err_pixel = criterion_l1(fake*multer, woutputv)
            #####################################
            D_G_z2 = output.data.mean()
            
            label.resize_(output.data.size())
            labelv = Variable(label.fill_(real_label))

            err_gan = argv.discriminator_weight*criterion_gan(output, labelv)
            
            ####################################
            #TODO normalize and minus mean?
            L,A,B=torch.chunk(fake,3,dim=1)
            LLL=torch.cat((L,L,L),1)
            out_feat=feat_model.features(LLL)
            
            #print(LLL.size())
            #break
            gt_feat = feat_model.features(outputv)
            
            gt_feat = gt_feat.detach() #don't require grad for this
            
            err_feat = argv.feature_weight * criterion_feat(out_feat,gt_feat)
            #err_feat.backward()
            
            # style loss
            Extract = FeatureExtractor(feat_model.features, ['0','5','10','19','28'])
            output_feature = Extract(LLL)
            target_feature = Extract(outputv)
            
            gram = GramMatrix()
            criterion_style = nn.MSELoss()
            err_style = 0
            gram_style = [gram(y) for y in target_feature]
            for m in range(len(output_feature)):
                gram_y = gram(output_feature[m])
                gram_s = Variable(gram_style[m].data, requires_grad=False)
                err_style += argv.style_weight * criterion_style(gram_y, gram_s)
                #style_loss = StyleLoss(gram_s, style_weight)
                        
            err_G = err_pixel + err_gan + err_feat + err_style
            err_G.backward()
            
            optimizerG.step()
            Loss_g_graph.append(err_G.data[0])
            Loss_gp_graph.append(err_pixel.data[0])
            Loss_gd_graph.append(err_gan.data[0])
            Loss_gf_graph.append(err_feat.data[0])
            # style loss
            Loss_gs_graph.append(err_style.data[0])
            #plt.imshow(vis_image(inputv.data.double().cpu()))

            print i, err_G.data[0]
            #TODO test on test set
            if(i%argv.visualize_every==0):
                test_img=clamp_image(fake.data.double().cpu())
                test_img=utforms.denormalize_lab(test_img)
                test_img=vis_image(test_img)
                test_img=(test_img*255).astype('uint8')
                test_img=np.transpose(test_img,(2,0,1))

                inp_img=vis_patch(utforms.denormalize_lab(img.cpu()),utforms.denormalize_lab(skg.cpu()))
                inp_img=(inp_img*255).astype('uint8')
                inp_img=np.transpose(inp_img,(2,0,1))
                
                target_img=vis_image(utforms.denormalize_lab(img.cpu()))
                target_img=(target_img*255).astype('uint8')
                target_img=np.transpose(target_img,(2,0,1))
                
                vis.image(test_img,win='output',opts=dict(title='output'))
                vis.image(inp_img,win='input',opts=dict(title='input'))  
                vis.image(inp_img,win='input',opts=dict(title='input'))
                vis.line(np.array(Loss_g_graph),win='g',opts=dict(title='G Total Loss'))
                vis.line(np.array(Loss_gd_graph),win='gd',opts=dict(title='G-Discriminator Loss'))
                vis.line(np.array(Loss_gf_graph),win='gf',opts=dict(title='G-Feature Loss'))
                vis.line(np.array(Loss_gs_graph),win='gs',opts=dict(title='G-Style Loss'))
                vis.line(np.array(Loss_gp_graph),win='gp',opts=dict(title='G-Pixel Loss'))
            
                
                vis.line(np.array(Loss_d_graph),win='d',opts=dict(title='D Loss'))

0 21.4126968384


/root/anaconda2/lib/python2.7/site-packages/skimage/color/colorconv.py:901: UserWarning: Color data out of range: Z < 0 in 3813 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)


1 24.0772075653
2 24.0121669769
3 23.7543716431
4 21.6208992004
5 17.1491641998
6 22.8539772034
7 16.5258312225
8 20.2977867126
9 17.4911785126
10 22.2822952271


/root/anaconda2/lib/python2.7/site-packages/skimage/color/colorconv.py:901: UserWarning: Color data out of range: Z < 0 in 1 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)


11 20.212480545
12 20.174118042
13 18.6332530975
14 25.1153335571
15 20.9299201965
16 24.0519962311
17 21.8115139008
18 14.9830856323
19 19.9711818695
20 19.1815357208


...

61 15.20246315
62 17.7686691284
63 19.4239959717
64 16.2610263824
65 22.0398178101
66 15.9951057434
67 18.5358905792
68 20.8865737915
69 20.2836265564
70 15.1854791641
71 18.8419723511
72 18.0091838837
73 18.8745136261
74 17.6629447937
75 16.3182754517
76 17.482831955
77 16.8110580444
78 19.0743179321
79 17.9721355438
80 15.4334478378


/root/anaconda2/lib/python2.7/site-packages/skimage/color/colorconv.py:901: UserWarning: Color data out of range: Z < 0 in 299 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)


81 18.6007900238
82 19.2045688629
83 16.5678958893
84 17.4924869537
85 13.7716026306
86 19.1496582031
87 22.1177711487
88 17.5926609039
89 22.0565681458
90 19.2117881775


/root/anaconda2/lib/python2.7/site-packages/skimage/color/colorconv.py:901: UserWarning: Color data out of range: Z < 0 in 25 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)


91 19.8318996429
92 16.1573581696
93 19.6679954529
94 18.486825943
95 15.2837085724
96 18.3991775513
97 15.352938652
98 14.3260602951
99 16.0827598572
100 16.7892894745
101 20.0837039948
102 17.5425052643
103 20.249546051
104 18.5180244446
105 16.9989891052
106 22.5404510498
107 13.7253913879
108 18.2062282562
109 20.534330368
110 16.3250656128
111 20.5362586975
112 19.382932663
113 19.0372714996
114 18.9360904694
115 16.6633052826
116 19.2016353607
117 12.850361824
118 17.968788147
119 16.8492183685
120 19.064163208
121 22.6282348633
122 17.4111499786
123 16.7578659058
124 16.5664615631
125 16.8402061462
126 17.7516403198
127 15.4398889542
128 17.8932704926
129 15.2866754532
130 20.886220932
131 14.2254619598
132 19.6662750244
133 17.3578834534
134 16.718372345
135 14.6905632019
136 18.9433746338
137 14.9018859863
138 14.0670576096
139 16.3570556641
140 16.8117694855
141 14.4000883102
142 13.594332695
143 19.2913589478
144 19.7831821442


In [10]:
fake

Variable containing:
( 0 , 0 ,.,.) = 
  5.2526e-02 -4.6358e-02 -3.0972e-02  ...   3.2919e-02 -3.5773e-02 -9.2999e-04
  1.2375e-01  9.6466e-02  1.5060e-01  ...   1.8420e-01  1.4148e-01  8.3080e-02
 -1.3774e-02 -2.9316e-02 -9.1480e-02  ...   5.9784e-02 -1.3701e-01  3.2708e-03
                 ...                   ⋱                   ...                
  3.0224e-02  1.0472e-01 -7.7999e-02  ...   4.3517e-02  1.9117e-02  3.7342e-02
  8.1598e-02  3.0137e-02 -6.3211e-02  ...   7.5752e-02 -1.4505e-02  5.6988e-02
  3.5128e-03  7.0972e-03 -8.1644e-02  ...  -3.8383e-03  1.5490e-03  4.1043e-02

( 0 , 1 ,.,.) = 
 -2.1616e-02 -5.1254e-02 -1.0209e-02  ...  -5.8665e-04  9.7732e-03  4.0185e-02
  4.0909e-03 -2.9581e-02  1.7048e-02  ...  -3.4723e-02  1.8971e-03  6.0333e-03
  7.7079e-03 -3.3754e-02 -1.2899e-02  ...  -2.5652e-02  4.3348e-02  1.6027e-02
                 ...                   ⋱                   ...                
 -2.5196e-02 -4.2128e-02 -9.8289e-03  ...   4.6661e-03  1.7585e-02  5.0481e

In [63]:
outputv

Variable containing:
( 0 , 0 ,.,.) = 
  0.8333  0.8333  0.8333  ...   0.8333  0.8333  0.8333
  0.8333  0.8333  0.8333  ...   0.8333  0.8333  0.8333
  0.8333  0.8333  0.8333  ...   0.8333  0.8333  0.8333
           ...             ⋱             ...          
  0.8333  0.8333  0.8333  ...   0.8333  0.8333  0.8333
  0.8333  0.8333  0.8333  ...   0.8333  0.8333  0.8333
  0.8333  0.8333  0.8333  ...   0.8333  0.8333  0.8333

( 0 , 1 ,.,.) = 
 -0.0000 -0.0000 -0.0000  ...  -0.0000 -0.0000 -0.0000
 -0.0000 -0.0000 -0.0000  ...  -0.0000 -0.0000 -0.0000
 -0.0000 -0.0000 -0.0000  ...  -0.0000 -0.0000 -0.0000
           ...             ⋱             ...          
 -0.0000 -0.0000 -0.0000  ...  -0.0000 -0.0000 -0.0000
 -0.0000 -0.0000 -0.0000  ...  -0.0000 -0.0000 -0.0000
 -0.0000 -0.0000 -0.0000  ...  -0.0000 -0.0000 -0.0000

( 0 , 2 ,.,.) = 
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000

In [15]:
def renormalize(img):
   
    #Renormalizes the input image to meet requirements for VGG-19 pretrained network
    

    forward_norm = torch.ones(img.data.size())*0.5
    forward_norm = Variable(forward_norm.cuda())
    img=img*(forward_norm)+forward_norm
    mean = img.data.new(img.data.size())
    std = img.data.new(img.data.size())
    mean[:, 0, :, :] = 0.485
    mean[:, 1, :, :] = 0.456
    mean[:, 2, :, :] = 0.406
    std[:, 0, :, :] = 0.229
    std[:, 1, :, :] = 0.224
    std[:, 2, :, :] = 0.225
    img -= Variable(mean)
    img = img / Variable(std)

    return img

In [51]:
Extract = FeatureExtractor(feat_model.features, ['0','5','10','19','28'])

In [61]:
Extract._modules.items()

[('submodule', Sequential (
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU (inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU (inplace)
    (4): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU (inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU (inplace)
    (9): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU (inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU (inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU (inplace)
    (18): MaxPool2d (size=(2, 2)

In [62]:
output_feature = Extract(LLL)
print(output_feature)

[Variable containing:
( 0 , 0 ,.,.) = 
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
           ...             ⋱             ...          
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000

( 0 , 1 ,.,.) = 
  0.4337  0.6208  0.4520  ...   0.6727  0.7107  0.5397
  0.1249  0.2651  0.0000  ...   0.2137  0.2436  0.1955
  0.2756  0.3664  0.3000  ...   0.3345  0.5373  0.3816
           ...             ⋱             ...          
  0.1609  0.5469  0.3859  ...   0.4039  0.3912  0.3041
  0.0976  0.4862  0.1902  ...   0.2604  0.2771  0.2918
  0.2070  0.4858  0.2189  ...   0.2944  0.2582  0.2315

( 0 , 2 ,.,.) = 
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.000

In [64]:
target_feature = Extract(outputv)
print(target_feature)

[Variable containing:
( 0 , 0 ,.,.) = 
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
           ...             ⋱             ...          
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000

( 0 , 1 ,.,.) = 
  0.0000  0.8004  0.8004  ...   0.8004  0.8004  1.3249
  0.0000  0.3457  0.3457  ...   0.3457  0.3457  1.2881
  0.0000  0.3457  0.3457  ...   0.3457  0.3457  1.2881
           ...             ⋱             ...          
  0.0000  0.3457  0.3457  ...   0.3457  0.3457  1.2881
  0.0000  0.3457  0.3457  ...   0.3457  0.3457  1.2881
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.4412

( 0 , 2 ,.,.) = 
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.000

In [42]:
class StyleLoss(nn.Module):

    def __init__(self, target, weight):
        super(StyleLoss, self).__init__()
        self.target = target.detach() * weight
        self.weight = weight
        self.gram = GramMatrix()
        self.criterion = nn.MSELoss()

    def forward(self, input):
        self.output = input.clone()
        self.G = self.gram(input)
        self.G.mul_(self.weight)
        self.loss = self.criterion(self.G, self.target)
        return self.output

    def backward(self, retain_variables=True):
        self.loss.backward(retain_variables=retain_variables)
        return self.loss

In [78]:
gram = GramMatrix()
mse_loss = nn.MSELoss()
style_error = 0

gram_style = [gram(y) for y in target_feature]
for m in range(len(output_feature)):
    gram_y = gram(output_feature[m])
    gram_s = Variable(gram_style[m].data, requires_grad=False)
    style_weight = 1
    style_error += style_weight * mse_loss(gram_y, gram_s)
    #style_loss = StyleLoss(gram_s, style_weight)

print(style_error)

Variable containing:
1.00000e-04 *
  1.2871
[torch.cuda.FloatTensor of size 1 (GPU 3)]

